In [1]:
!pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 2.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357280 sha256=e3b1bfe1f23633ef56ff397aed981141bfebb47d248444191c2e8d38b331e9df
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


In [2]:
!curl -o dataset.zip "https://files.grouplens.org/datasets/movielens/ml-1m.zip"
!unzip dataset.zip
!ls -la

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 5778k  100 5778k    0     0  8607k      0 --:--:-- --:--:-- --:--:-- 8612k
Archive:  dataset.zip
   creating: ml-1m/
  inflating: ml-1m/movies.dat        
  inflating: ml-1m/ratings.dat       
  inflating: ml-1m/README            
  inflating: ml-1m/users.dat         
total 5800
drwxr-xr-x 1 root root    4096 Oct 29 00:33 .
drwxr-xr-x 1 root root    4096 Oct 29 00:29 ..
drwxr-xr-x 4 root root    4096 Oct 25 13:20 .config
-rw-r--r-- 1 root root 5917549 Oct 29 00:33 dataset.zip
drwxr-x--- 2 root root    4096 Jan 29  2016 ml-1m
drwxr-xr-x 1 root root    4096 Oct 25 13:20 sample_data


In [3]:
import pandas as pd

In [4]:
ratings = pd.read_csv('ml-1m/ratings.dat', sep='::', header=None, engine='python',
                      names=['userId', 'movieId', 'rating', 'timestamp'], encoding='latin-1')
movies  = pd.read_csv('ml-1m/movies.dat', sep='::',  header=None, engine='python',
                      names=['movieId', 'title', 'genres'], encoding='latin-1')
users   = pd.read_csv('ml-1m/users.dat', sep='::',  header=None, engine='python',
                      names=['userId', 'gender', 'age', 'occupation', 'zip-code'], encoding='latin-1')

# Pregunta 1

KNN es un modelo que requiere de una métrica de similitud para reconocer y ordenar a los vecinos más cercanos. Es comun usar la distancia euclidiana en contextos de ML sin embargo para un enfoque de sistemas de recomendación una métrica de similitud coseno o RMSE es más habitual.

Ventajas:
* Proceso de finetuning sencillo

Limitaciones:
* No es la mejor escalando a grandes conjuntos de datos
* Sensible a las métricas de similitud

# Pregunta 2



In [5]:
ratings.head(2)

,userId,movieId,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109


In [8]:
ratings.rating.unique()

array([5, 3, 4, 2, 1])

In [17]:
films_user_1 = set(ratings[ratings['userId'] == 1]['movieId'].to_list())
films_user_2 = set(ratings[ratings['userId'] == 2]['movieId'].to_list())

In [18]:
films_user_1.intersection(films_user_2)

{1193, 1207, 1246, 1962, 2028, 2321, 3105}

In [56]:
user_movie_rating = ratings.pivot(index='userId', columns='movieId', values='rating')

In [23]:
user_movie_rating.head(2)

movieId,1,2,3,4,5,6,7,8,9,10,...,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
userId,,,,,,,,,,,,,,,,,,,,,
1,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [60]:
user_1_2 = user_movie_rating[list(films_user_1.intersection(films_user_2))].loc[[1, 2]]

In [62]:
user_1_2.T.corr()

userId,1,2
userId,,
1,1.000000,0.416667
2,0.416667,1.000000


En este caso tenemos una correlación menor a 0.5 lo cual es una correlacion baja. Esto nos da indicios de que no hay mucha similitud de gustos por las películas entre el usuario 1 y el usuario 2.

# Pregunta 3

In [53]:
films_g50 = ratings.groupby('movieId').count()['rating']
films_g50

,rating
movieId,
1,2077
2,701
3,478
4,170
5,296
...,...
3948,862
3949,304
3950,54


In [57]:
from sklearn.metrics.pairwise import cosine_similarity
# user_movie_rating[[1,2,3]]
question_3 = user_movie_rating[[1,2,3]].dropna(axis=0)
question_3

movieId,1,2,3
userId,,,
163,4.0,3.0,1.0
223,5.0,5.0,4.0
301,5.0,3.0,4.0
308,4.0,4.0,3.0
310,4.0,4.0,3.0
...,...,...,...
5643,5.0,3.0,2.0
5720,3.0,1.0,1.0
5747,5.0,4.0,2.0


In [58]:
movie_similarity = cosine_similarity(question_3.T)
movie_similarity

array([[1.        , 0.95335877, 0.943227  ],
       [0.95335877, 1.        , 0.92178349],
       [0.943227  , 0.92178349, 1.        ]])

* Se puede hacer un ranking de similitud para mostrar las películas con distancia coseno más altas ordenadas decrecientemente.

* Tambien dada una película se puede recuperar aquella que tiene similitud coseno más cercana a 1.

* Adicionalmente, siguiendo un enfoque de diversidad es importante también recomendar algunas películas que estén un poco más abajo en el ranking de similitud



Pregunta